In [20]:
import requests
from datetime import datetime, timedelta, date
import pandas as pd
import numpy as np

import apimoex

In [21]:
def get_current_sec_id(ticker: str, trade_date) -> str:
    """
    Функция обращается к MOEX API и возвращает актуальный sec_id фьючерса на запрошенный инструмент и дату
    :param ticker: str
    :return: str
    """
    request_url = (f'http://iss.moex.com/iss/engines/futures/markets/forts/securities.json?assets={ticker}')
    arguments = {'securities.columns': ('SECID, LASTTRADEDATE, LASTDELDATE, SHORTNAME, ASSETCODE')}
    with requests.Session() as session:
        iss = apimoex.ISSClient(session, request_url, arguments)
        data = iss.get()
        df = pd.DataFrame(data['securities'])

    df[['LASTTRADEDATE', 'LASTDELDATE']] = df[['LASTTRADEDATE', 'LASTDELDATE']].apply(pd.to_datetime)
    min_date = df['LASTTRADEDATE'].min()
    sec_id = (
        df.query('LASTTRADEDATE > @trade_date')
        .query('LASTTRADEDATE == @min_date')
        .reset_index(drop=True)
        .loc[0, 'SECID']
    )

    return sec_id

Запрос актуального фьючерса на дату

In [22]:
trade_date: date = datetime.strptime('2024-08-19', "%Y-%m-%d").date()  # Дату можно изменить
ticker: str = 'RTS'  # Задание класса фьючерсов

sec_id: str = get_current_sec_id(ticker, trade_date)  # Запрос на актуальный фьючерс
print(sec_id)

RIU4


In [23]:
def get_info_security(session, security: str):
    """
    Запрашивает у MOEX информацию по инструменту
    :param session: Подключение к MOEX
    :param security: Тикер инструмента
    :return: Дата последних торгов бара (если её нет то возвращает 2130.01.01), страйк, тип опциона, код фьючерса
    """
    security_info = apimoex.find_security_description(session, security)
    df = pd.DataFrame(security_info)  # Полученную информацию по тикеру в DF
    name_lst = list(df['name'])
    if 'LSTTRADE' in name_lst:
        row_lst = ['NAME', 'LSTTRADE', 'OPTIONTYPE', 'STRIKE']  # Список необходимых строк из DF
        # Меняем значение в ячейке ['NAME', 'value'] на конец строки 'value', где прописан код фьючерса
        df.loc[df[df['name'] == 'NAME'].index, 'value'] = list(df.loc[df[df['name'] == 'NAME'].index]['value'])[
            0].split().pop()
        df = df[df['name'].isin(row_lst)]  # Выборка необходимых строк
        rez_lst: list = list(df.value)  # Колонку 'value' в список
        return pd.Series(rez_lst)
    else:
        return pd.Series([float('nan'), '2130-01-01', float('nan'), float('nan')])


In [24]:

def get_options_date_results(tradedate: date, shortname: str):
    # tradedate = pd.to_datetime(tradedate).date
    df_rez = pd.DataFrame()
    arguments = {'securities.columns': (
        "BOARDID, TRADEDATE, SECID, OPEN, LOW, HIGH, CLOSE, OPENPOSITIONVALUE, VALUE, VOLUME, OPENPOSITION, SETTLEPRICE"
    )}
    # arguments = {'securities.columns': ("TRADEDATE, SECID, OPENPOSITION")}

    with requests.Session() as session:
        page = 0  # С какой записи стартовать запрос
        while True:  # В цикле отправляем запрос постранично и обрабатываем ответ
            # request_url = (f'http://iss.moex.com/iss/history/engines/futures/markets/options/securities.json?'
            #                f'date={tradedate}&assets=RTS&start={page}')
            request_url = (f'http://iss.moex.com/iss/history/engines/futures/markets/options/securities.json?'
                           f'date={tradedate}&assets={shortname}&start={page}')
            print(f'{request_url=}')
            iss = apimoex.ISSClient(session, request_url, arguments)
            data = iss.get()
            df = pd.DataFrame(data['history'])  # Полученные исторические данные в DF
            # print(df.to_string(max_rows=20, max_cols=15), '\n')
            if len(df) == 0:  # Больше нет страниц в ответе
                break
            else:
                df = df[["TRADEDATE", "SECID", "OPENPOSITION"]]  # Оставляем нужные поля
                # df = df.drop(columns=['BOARDID', 'SETTLEPRICEDAY', 'WAPRICE'])  # Удаляем не нужные поля
                # Создаем новые колонки 'NAME', 'LSTTRADE', 'OPTIONTYPE', 'STRIKE' и заполняем
                df[['NAME', 'LSTTRADE', 'OPTIONTYPE', 'STRIKE']] = df.apply(
                    lambda x: get_info_security(session, x['SECID']), axis=1)
                # print(df.to_string(max_rows=20, max_cols=15), '\n')
                
                # # Меняем формат колонки на дату
                # df["LSTTRADE"] = pd.to_datetime(df["LSTTRADE"]).date
                # # df["LSTTRADE"] = np.datetime64(df["LSTTRADE"])
                
                # # Оставляем только строки, где дата экспирации опциона больше даты бара фьючерса(исключаем ОИ=0)
                # df = df.loc[df['LSTTRADE'] > tradedate]
                
                df = df.loc[df['NAME'] == shortname]  # Выбор опционов текущего базового актива
                # Заполняем пропущенные значения в столбце OPENPOSITION значением 0.0
                df['OPENPOSITION'] = df['OPENPOSITION'].fillna(0.0)
                # df = df[df['LSTTRADE'] == df['LSTTRADE'].min()]  # Выборка строк с минимальной датой
                # df_rez = pd.concat([df_rez, df]).reset_index(drop=True)  # Слияние DF
                df_rez = pd.concat([df_rez.dropna(), df.dropna()]).reset_index(drop=True)
                print(df_rez.to_string(max_rows=6, max_cols=18), '\n')
                page += 100  # для запроса следующей страницы со 100 записями

    return df_rez

http://iss.moex.com/iss/engines/futures/markets/options/securities.json?date=2024-08-19&assets=RTS&start=0

In [25]:
df = get_options_date_results(trade_date, sec_id)  # Получаем DF по опционам от МОЕХ
df

request_url='http://iss.moex.com/iss/history/engines/futures/markets/options/securities.json?date=2024-08-19&assets=RIU4&start=0'
Empty DataFrame
Columns: [TRADEDATE, SECID, OPENPOSITION, NAME, LSTTRADE, OPTIONTYPE, STRIKE]
Index: [] 

request_url='http://iss.moex.com/iss/history/engines/futures/markets/options/securities.json?date=2024-08-19&assets=RIU4&start=100'
Empty DataFrame
Columns: [TRADEDATE, SECID, OPENPOSITION, NAME, LSTTRADE, OPTIONTYPE, STRIKE]
Index: [] 

request_url='http://iss.moex.com/iss/history/engines/futures/markets/options/securities.json?date=2024-08-19&assets=RIU4&start=200'
Empty DataFrame
Columns: [TRADEDATE, SECID, OPENPOSITION, NAME, LSTTRADE, OPTIONTYPE, STRIKE]
Index: [] 

request_url='http://iss.moex.com/iss/history/engines/futures/markets/options/securities.json?date=2024-08-19&assets=RIU4&start=300'
Empty DataFrame
Columns: [TRADEDATE, SECID, OPENPOSITION, NAME, LSTTRADE, OPTIONTYPE, STRIKE]
Index: [] 

request_url='http://iss.moex.com/iss/history/engin

KeyboardInterrupt: 

https://iss.moex.com/iss/history/engines/futures/markets/options/securities/RI100000BH4D/candles.json?interval=1  
https://iss.moex.com/iss/engines/futures/markets/options/securities/RI100000BH4D/candles.json?interval=1  
https://iss.moex.com/iss/engines/futures/markets/options/securities/RI100000BH4D/candles.json?interval=1&2024-08-19